In [1]:
import pandas as pd
import os
import glob
from pathlib import Path

# 데이터 폴더 경로 설정
data_folder = '../data'

# 대학교 관련 csv 파일들 찾기
university_csv_files = [
    '전북대_금암동18-1.csv',
    '전북대_금암동634-18.csv', 
    '전북대_금암동663.csv',
    '전북대_덕진동1가664-1.csv',
    '서울대_신림동56-1.csv',
    '서울대_봉천동4-2.csv',
    '서울대_봉천동4-1.csv',
    '부경대_559-1.csv',
    '부산대_장전동40.csv',
    '부산대_장전동30.csv',
    '경북대학교.csv'
]

print(f"발견된 대학교 csv 파일들: {len(university_csv_files)}개")
for file in university_csv_files:
    print(f"- {file}")

발견된 대학교 csv 파일들: 11개
- 전북대_금암동18-1.csv
- 전북대_금암동634-18.csv
- 전북대_금암동663.csv
- 전북대_덕진동1가664-1.csv
- 서울대_신림동56-1.csv
- 서울대_봉천동4-2.csv
- 서울대_봉천동4-1.csv
- 부경대_559-1.csv
- 부산대_장전동40.csv
- 부산대_장전동30.csv
- 경북대학교.csv


In [2]:
# 모든 대학교 csv 파일들을 읽어서 통합
all_dataframes = []
file_info = []

for file in university_csv_files:
    file_path = os.path.join(data_folder, file)
    
    try:
        # csv 파일 읽기 (인코딩 문제 대비)
        try:
            df = pd.read_csv(file_path, encoding='utf-8')
        except UnicodeDecodeError:
            df = pd.read_csv(file_path, encoding='cp949')
        
        # 파일명에서 대학교명과 지역 정보 추출
        university_name = file.split('_')[0]
        location_info = file.replace('.csv', '').split('_')[1] if '_' in file else ''
        
        # 데이터프레임에 출처 정보 추가
        df['대학교명'] = university_name
        df['지역정보'] = location_info
        df['원본파일명'] = file
        
        all_dataframes.append(df)
        
        file_info.append({
            '파일명': file,
            '대학교명': university_name,
            '지역정보': location_info,
            '데이터수': len(df),
            '컬럼수': len(df.columns)
        })
        
        print(f"✓ {file}: {len(df)}건 로드 완료")
        
    except Exception as e:
        print(f"✗ {file} 읽기 실패: {str(e)}")

# 파일 정보 요약
file_info_df = pd.DataFrame(file_info)
print("\n=== 파일 로드 요약 ===")
print(file_info_df)

✓ 전북대_금암동18-1.csv: 17건 로드 완료
✓ 전북대_금암동634-18.csv: 31건 로드 완료
✓ 전북대_금암동663.csv: 31건 로드 완료
✓ 전북대_덕진동1가664-1.csv: 106건 로드 완료
✓ 서울대_신림동56-1.csv: 152건 로드 완료
✓ 서울대_봉천동4-2.csv: 23건 로드 완료
✓ 서울대_봉천동4-1.csv: 15건 로드 완료
✓ 부경대_559-1.csv: 58건 로드 완료
✓ 부산대_장전동40.csv: 88건 로드 완료
✓ 부산대_장전동30.csv: 22건 로드 완료
✓ 경북대학교.csv: 118건 로드 완료

=== 파일 로드 요약 ===
                   파일명       대학교명        지역정보  데이터수  컬럼수
0      전북대_금암동18-1.csv        전북대     금암동18-1    17   80
1    전북대_금암동634-18.csv        전북대   금암동634-18    31   80
2       전북대_금암동663.csv        전북대      금암동663    31   80
3   전북대_덕진동1가664-1.csv        전북대  덕진동1가664-1   106   80
4      서울대_신림동56-1.csv        서울대     신림동56-1   152   80
5       서울대_봉천동4-2.csv        서울대      봉천동4-2    23   80
6       서울대_봉천동4-1.csv        서울대      봉천동4-1    15   80
7        부경대_559-1.csv        부경대       559-1    58   80
8        부산대_장전동40.csv        부산대       장전동40    88   80
9        부산대_장전동30.csv        부산대       장전동30    22   80
10           경북대학교.csv  경북대학교.csv          

In [3]:
# 모든 데이터프레임을 하나로 통합
if all_dataframes:
    combined_df = pd.concat(all_dataframes, ignore_index=True)
    
    print(f"=== 통합 결과 ===")
    print(f"총 레코드 수: {len(combined_df):,}건")
    print(f"총 컬럼 수: {len(combined_df.columns)}개")
    print(f"통합된 대학교 수: {combined_df['대학교명'].nunique()}개")
    
    # 대학교별 건수 확인
    print("\n=== 대학교별 건수 ===")
    university_counts = combined_df['대학교명'].value_counts()
    print(university_counts)
    
    # 지역별 건수 확인
    print("\n=== 지역별 건수 ===")
    location_counts = combined_df.groupby(['대학교명', '지역정보']).size().sort_values(ascending=False)
    print(location_counts)
    
    # 통합된 데이터 저장
    output_file = os.path.join(data_folder, '대학교_통합_데이터.csv')
    combined_df.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"\n통합 데이터가 저장되었습니다: {output_file}")
    
    # 엑셀로도 저장
    output_excel = os.path.join(data_folder, '대학교_통합_데이터.xlsx')
    combined_df.to_excel(output_excel, index=False)
    print(f"엑셀 파일도 저장되었습니다: {output_excel}")
    
    # 기본 정보 출력
    print(f"\n=== 통합 데이터 기본 정보 ===")
    print(f"전체 행 수: {len(combined_df):,}")
    print(f"전체 열 수: {len(combined_df.columns)}")
    print(f"메모리 사용량: {combined_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # 첫 몇 행 확인
    print(f"\n=== 데이터 샘플 (처음 3행) ===")
    display_cols = ['대지위치', '건물명', '연면적(㎡)', '주용도코드명', '대학교명', '지역정보']
    print(combined_df[display_cols].head(3))
    
else:
    print("로드된 데이터프레임이 없습니다.")

=== 통합 결과 ===
총 레코드 수: 661건
총 컬럼 수: 80개
통합된 대학교 수: 5개

=== 대학교별 건수 ===
대학교명
서울대          190
전북대          185
경북대학교.csv    118
부산대          110
부경대           58
Name: count, dtype: int64

=== 지역별 건수 ===
대학교명       지역정보      
서울대        신림동56-1       152
경북대학교.csv                118
전북대        덕진동1가664-1    106
부산대        장전동40          88
부경대        559-1          58
전북대        금암동663         31
           금암동634-18      31
서울대        봉천동4-2         23
부산대        장전동30          22
전북대        금암동18-1        17
서울대        봉천동4-1         15
dtype: int64

통합 데이터가 저장되었습니다: ../data\대학교_통합_데이터.csv
엑셀 파일도 저장되었습니다: ../data\대학교_통합_데이터.xlsx

=== 통합 데이터 기본 정보 ===
전체 행 수: 661
전체 열 수: 80
메모리 사용량: 1.45 MB

=== 데이터 샘플 (처음 3행) ===
                          대지위치  건물명    연면적(㎡)  주용도코드명 대학교명     지역정보
0   전북특별자치도 전주시 덕진구 금암동 18-1번지  NaN   5046.47  교육연구시설  전북대  금암동18-1
1  전북특별자치도 전주시 덕진구 금암동 산18-1번지  NaN  20781.85  교육연구시설  전북대  금암동18-1
2  전북특별자치도 전주시 덕진구 금암동 산18-1번지  NaN   4654.80  교육연구시설  전북대  금암동18-1


In [4]:
combined_df

,대지위치,시군구코드,법정동코드,대지구분코드,번,지,관리건축물대장PK,대장구분코드,대장구분코드명,대장종류코드,...,친환경건축물등급,친환경건축물인증점수,지능형건축물등급,지능형건축물인증점수,생성일자,내진설계적용여부,내진능력,대학교명,지역정보,원본파일명
0,전북특별자치도 전주시 덕진구 금암동 18-1번지,52113,10700,0,18,1,11861132383,1,일반,2,...,NaN,0.0,NaN,0,20220924,0,NaN,전북대,금암동18-1,전북대_금암동18-1.csv
1,전북특별자치도 전주시 덕진구 금암동 산18-1번지,52113,10700,1,18,1,11861136862,1,일반,2,...,NaN,0.0,NaN,0,20220924,0,NaN,전북대,금암동18-1,전북대_금암동18-1.csv
2,전북특별자치도 전주시 덕진구 금암동 산18-1번지,52113,10700,1,18,1,11861136848,1,일반,2,...,NaN,0.0,NaN,0,20220924,0,NaN,전북대,금암동18-1,전북대_금암동18-1.csv
3,전북특별자치도 전주시 덕진구 금암동 산18-1번지,52113,10700,1,18,1,11861136847,1,일반,2,...,NaN,0.0,NaN,0,20220924,0,NaN,전북대,금암동18-1,전북대_금암동18-1.csv
4,전북특별자치도 전주시 덕진구 금암동 산18-1번지,52113,10700,1,18,1,11861136852,1,일반,2,...,NaN,0.0,NaN,0,20220924,0,NaN,전북대,금암동18-1,전북대_금암동18-1.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656,대구광역시 북구 산격동 1370-1번지,27230,11100,0,1370,1,10491100256825,1,일반,2,...,NaN,0.0,NaN,0,20221126,0,NaN,경북대학교.csv,,경북대학교.csv
657,대구광역시 북구 산격동 1370-1번지,27230,11100,0,1370,1,1049122198,1,일반,2,...,NaN,0.0,NaN,0,20221126,0,NaN,경북대학교.csv,,경북대학교.csv
658,대구광역시 북구 산격동 1370-1번지,27230,11100,0,1370,1,1049122295,1,일반,2,...,NaN,0.0,NaN,0,20221126,0,NaN,경북대학교.csv,,경북대학교.csv
659,대구광역시 북구 산격동 1370-1번지,27230,11100,0,1370,1,1049122200,1,일반,2,...,NaN,0.0,NaN,0,20221126,0,NaN,경북대학교.csv,,경북대학교.csv


In [5]:
# 1. 관리건축물대장PK 컬럼명을 new_MGM으로 변경
combined_df = combined_df.rename(columns={'관리건축물대장PK': 'new_MGM'})

print("1. 컬럼명 변경 완료: 관리건축물대장PK → new_MGM")

# 2. MGM_BLD_PK 변수 생성
def create_mgm_bld_pk(row):
    시군구코드 = str(row['시군구코드'])
    new_mgm = str(row['new_MGM'])
    
    # new_MGM 길이가 20 이상이면 그대로 사용
    if len(new_mgm) >= 20:
        processed_mgm = new_mgm
    else:
        # 20 미만이면 앞 5자리 제거 (길이가 5자리 미만인 경우 그대로 사용)
        processed_mgm = new_mgm[5:] if len(new_mgm) > 5 else new_mgm
    
    return f"{시군구코드}-{processed_mgm}"

# MGM_BLD_PK 변수 생성
combined_df['MGM_BLD_PK'] = combined_df.apply(create_mgm_bld_pk, axis=1)

print("2. MGM_BLD_PK 변수 생성 완료")

# 결과 확인
print("\n=== 변수 생성 결과 확인 ===")
print("new_MGM 길이 분포:")
mgm_lengths = combined_df['new_MGM'].astype(str).str.len()
print(mgm_lengths.value_counts().sort_index())

print(f"\n샘플 데이터 (처음 5행):")
sample_cols = ['시군구코드', 'new_MGM', 'MGM_BLD_PK', '대학교명']
print(combined_df[sample_cols].head())

print(f"\n각 대학교별 MGM_BLD_PK 길이 분포:")
for univ in combined_df['대학교명'].unique():
    univ_data = combined_df[combined_df['대학교명'] == univ]
    mgm_bld_lengths = univ_data['MGM_BLD_PK'].str.len()
    print(f"{univ}: 평균 길이 {mgm_bld_lengths.mean():.1f}, 범위 {mgm_bld_lengths.min()}-{mgm_bld_lengths.max()}")

1. 컬럼명 변경 완료: 관리건축물대장PK → new_MGM
2. MGM_BLD_PK 변수 생성 완료

=== 변수 생성 결과 확인 ===
new_MGM 길이 분포:
new_MGM
9     236
10     91
11    100
14    215
22     19
Name: count, dtype: int64

샘플 데이터 (처음 5행):
   시군구코드      new_MGM    MGM_BLD_PK 대학교명
0  52113  11861132383  52113-132383  전북대
1  52113  11861136862  52113-136862  전북대
2  52113  11861136848  52113-136848  전북대
3  52113  11861136847  52113-136847  전북대
4  52113  11861136852  52113-136852  전북대

각 대학교별 MGM_BLD_PK 길이 분포:
전북대: 평균 길이 13.5, 범위 12-28
서울대: 평균 길이 12.0, 범위 10-28
부경대: 평균 길이 12.5, 범위 10-28
부산대: 평균 길이 12.0, 범위 10-28
경북대학교.csv: 평균 길이 12.7, 범위 11-28


In [18]:
# 매칭용 엑셀 파일 불러오기
mc_df = pd.read_excel('../data/batch_stage_matching_result_final.xlsx')

print(f"매칭 데이터 로드 완료: {len(mc_df):,}건")
print(f"컬럼 수: {len(mc_df.columns)}개")
print(f"주요 컬럼: {list(mc_df.columns[:10])}")  # 처음 10개 컬럼만 출력
print(f"\n데이터 샘플:")
print(mc_df.head(3))

매칭 데이터 로드 완료: 4,194건
컬럼 수: 48개
주요 컬럼: ['SEQ_NO', 'RECAP_PK', '연면적', '사용승인연도', '기관명', '건축물명', 'new_주소', '주소', '지상', '지하']

데이터 샘플:
   SEQ_NO         RECAP_PK       연면적     사용승인연도              기관명     건축물명  \
0  392491              NaN   5893.03 1999-05-26  강원특별자치도 강원도립대학교   기숙사-B동   
1  214137  51150-100183531   4467.30 2012-07-09      강원특별자치도 강릉시   생활체육센터   
2  214178  51150-100183531  17123.77 1998-12-31      강원특별자치도 강릉시  실내종합체육관   

                                      new_주소  \
0                     강원 강릉시 주문진읍 교항리 48-160   
1    강원특별자치도 강릉시 (교동 2-10번지, 생활체육센터 ) 생활체육센터   
2  강원특별자치도 강릉시 (교동 2-10번지, 실내종합체육관 ) 실내종합체육관   

                                          주소   지상   지하  ...  \
0                     강원 강릉시 주문진읍 교항리 48-160  4.0  1.0  ...   
1    강원특별자치도 강릉시 (교동 2-10번지, 생활체육센터 ) 생활체육센터  3.0  NaN  ...   
2  강원특별자치도 강릉시 (교동 2-10번지, 실내종합체육관 ) 실내종합체육관  4.0  1.0  ...   

   건축물 에너지사용량 입력자 연락처 EUI(1차E/냉난방 면적) EUI(1차E/연면적)   ebd_tokens    연면적_원본  \
0        033-660-8041       85.173567 

In [7]:
# mc_df에서 대학교 관련 기관명 확인
print("=== mc_df의 기관명에서 대학교 관련 데이터 확인 ===")

# 대학교 키워드가 포함된 기관명들 찾기
university_related = mc_df[mc_df['기관명'].str.contains('대학', na=False)]
unique_institutions = university_related['기관명'].unique()

print(f"대학교 관련 기관 수: {len(unique_institutions)}개")
print("\n실제 기관명들:")
for inst in sorted(unique_institutions):
    count = len(mc_df[mc_df['기관명'] == inst])
    print(f"- {inst}: {count}건")
    

=== mc_df의 기관명에서 대학교 관련 데이터 확인 ===
대학교 관련 기관 수: 61개

실제 기관명들:
- 강릉원주대학교: 25건
- 강릉원주대학교치과병원: 1건
- 강원대학교: 73건
- 강원대학교병원: 1건
- 강원특별자치도 강원도립대학교: 3건
- 경남도립거창대학: 4건
- 경남도립남해대학: 5건
- 경북대학교: 73건
- 경북대학교병원: 6건
- 경북대학교치과병원: 1건
- 경북도립대학교: 3건
- 경상국립대학교: 59건
- 경상국립대학교병원: 5건
- 경인교육대학교: 16건
- 공주교육대학교: 5건
- 공주대학교: 37건
- 광주교육대학교: 10건
- 국립금오공과대학교: 13건
- 국립한밭대학교: 16건
- 군산대학교: 23건
- 대구교육대학교: 6건
- 목포대학교: 26건
- 목포해양대학교: 8건
- 부경대학교: 35건
- 부산교육대학교: 8건
- 부산대학교: 56건
- 부산대학교병원: 5건
- 서울과학기술대학교: 19건
- 서울교육대학교: 7건
- 서울대학교: 124건
- 서울대학교병원: 10건
- 서울대학교치과병원: 3건
- 서울시립대학교: 16건
- 순천대학교: 23건
- 안동대학교: 26건
- 인천대학교: 5건
- 전남대학교: 29건
- 전남대학교병원: 8건
- 전남도립대학교: 2건
- 전북대학교: 66건
- 전북대학교병원: 11건
- 전주교육대학교: 6건
- 제주대학교: 37건
- 진주교육대학교: 10건
- 창원대학교: 26건
- 청주교육대학교: 6건
- 춘천교육대학교: 6건
- 충남대학교: 36건
- 충남대학교병원: 6건
- 충남도립대학교: 5건
- 충북대학교: 37건
- 충북대학교병원: 3건
- 충북도립대학: 4건
- 한경국립대학교: 13건
- 한국교원대학교: 27건
- 한국교통대학교: 23건
- 한국기술교육대학교: 23건
- 한국방송통신대학교: 20건
- 한국전통문화대학교: 7건
- 한국체육대학교: 7건
- 한국해양대학교: 15건


In [19]:
# 매칭 대상 대학교들
target_universities = ['서울대학교', '전북대학교', '부경대학교', '경북대학교', '부산대학교']

print("=== 각 대학교별 MGM_BLD_PK 비어있는 건수 확인 ===")

total_empty = 0
university_summary = []

for univ_name in target_universities:
    # 해당 대학교 데이터 필터링
    univ_data = mc_df[mc_df['기관명'] == univ_name]
    
    # MGM_BLD_PK가 비어있는 건수 확인
    if 'MGM_BLD_PK' in mc_df.columns:
        empty_mgm = univ_data['MGM_BLD_PK'].isna() | (univ_data['MGM_BLD_PK'] == '')
    else:
        # MGM_BLD_PK 컬럼이 없으면 모든 데이터가 비어있다고 간주
        empty_mgm = pd.Series([True] * len(univ_data), index=univ_data.index)
    
    empty_count = empty_mgm.sum()
    total_count = len(univ_data)
    
    university_summary.append({
        '대학교': univ_name,
        '전체건수': total_count,
        '비어있는건수': empty_count,
        '비율': f"{empty_count/total_count*100:.1f}%" if total_count > 0 else "0%"
    })
    
    total_empty += empty_count
    
    print(f"{univ_name}: {empty_count}건 / {total_count}건 ({empty_count/total_count*100:.1f}%)")

print(f"\n총 매칭 대상: {total_empty}건")

# 요약 테이블
summary_df = pd.DataFrame(university_summary)
print(f"\n=== 요약 테이블 ===")
print(summary_df)

=== 각 대학교별 MGM_BLD_PK 비어있는 건수 확인 ===
서울대학교: 33건 / 124건 (26.6%)
전북대학교: 47건 / 66건 (71.2%)
부경대학교: 6건 / 35건 (17.1%)
경북대학교: 52건 / 73건 (71.2%)
부산대학교: 40건 / 56건 (71.4%)

총 매칭 대상: 178건

=== 요약 테이블 ===
     대학교  전체건수  비어있는건수     비율
0  서울대학교   124      33  26.6%
1  전북대학교    66      47  71.2%
2  부경대학교    35       6  17.1%
3  경북대학교    73      52  71.2%
4  부산대학교    56      40  71.4%


In [9]:
combined_df

,대지위치,시군구코드,법정동코드,대지구분코드,번,지,new_MGM,대장구분코드,대장구분코드명,대장종류코드,...,친환경건축물인증점수,지능형건축물등급,지능형건축물인증점수,생성일자,내진설계적용여부,내진능력,대학교명,지역정보,원본파일명,MGM_BLD_PK
0,전북특별자치도 전주시 덕진구 금암동 18-1번지,52113,10700,0,18,1,11861132383,1,일반,2,...,0.0,NaN,0,20220924,0,NaN,전북대,금암동18-1,전북대_금암동18-1.csv,52113-132383
1,전북특별자치도 전주시 덕진구 금암동 산18-1번지,52113,10700,1,18,1,11861136862,1,일반,2,...,0.0,NaN,0,20220924,0,NaN,전북대,금암동18-1,전북대_금암동18-1.csv,52113-136862
2,전북특별자치도 전주시 덕진구 금암동 산18-1번지,52113,10700,1,18,1,11861136848,1,일반,2,...,0.0,NaN,0,20220924,0,NaN,전북대,금암동18-1,전북대_금암동18-1.csv,52113-136848
3,전북특별자치도 전주시 덕진구 금암동 산18-1번지,52113,10700,1,18,1,11861136847,1,일반,2,...,0.0,NaN,0,20220924,0,NaN,전북대,금암동18-1,전북대_금암동18-1.csv,52113-136847
4,전북특별자치도 전주시 덕진구 금암동 산18-1번지,52113,10700,1,18,1,11861136852,1,일반,2,...,0.0,NaN,0,20220924,0,NaN,전북대,금암동18-1,전북대_금암동18-1.csv,52113-136852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656,대구광역시 북구 산격동 1370-1번지,27230,11100,0,1370,1,10491100256825,1,일반,2,...,0.0,NaN,0,20221126,0,NaN,경북대학교.csv,,경북대학교.csv,27230-100256825
657,대구광역시 북구 산격동 1370-1번지,27230,11100,0,1370,1,1049122198,1,일반,2,...,0.0,NaN,0,20221126,0,NaN,경북대학교.csv,,경북대학교.csv,27230-22198
658,대구광역시 북구 산격동 1370-1번지,27230,11100,0,1370,1,1049122295,1,일반,2,...,0.0,NaN,0,20221126,0,NaN,경북대학교.csv,,경북대학교.csv,27230-22295
659,대구광역시 북구 산격동 1370-1번지,27230,11100,0,1370,1,1049122200,1,일반,2,...,0.0,NaN,0,20221126,0,NaN,경북대학교.csv,,경북대학교.csv,27230-22200


In [10]:
# combined_df의 대학교명을 mc_df 형식으로 변경
university_name_mapping = {
    '서울대': '서울대학교',
    '전북대': '전북대학교', 
    '부경대': '부경대학교',
    '부산대': '부산대학교',
    '경북대학교.csv': '경북대학교'
}

combined_df['대학교명'] = combined_df['대학교명'].map(university_name_mapping)

print("대학교명 변경 완료:")
print(combined_df['대학교명'].value_counts())

대학교명 변경 완료:
대학교명
서울대학교    190
전북대학교    185
경북대학교    118
부산대학교    110
부경대학교     58
Name: count, dtype: int64


In [20]:
import re
from collections import defaultdict

def tokenize_text(text):
    """텍스트를 전처리하고 토큰화하여 집합(set)으로 반환"""
    if pd.isna(text) or text is None:
        return set()
    
    text = str(text).lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    tokens = text.split()
    tokens = [token.strip() for token in tokens if token.strip()]
    return set(tokens)

def safe_date_convert(date_value):
    """날짜를 YYYY-MM-DD 형식으로 안전하게 변환"""
    if pd.isna(date_value):
        return None
    try:
        if hasattr(date_value, 'strftime'):
            return date_value.strftime('%Y-%m-%d')
        return pd.to_datetime(date_value).strftime('%Y-%m-%d')
    except:
        return None

def safe_equals(a, b):
    """안전한 동등 비교"""
    if pd.isna(a) or pd.isna(b):
        return False
    try:
        return float(a) == float(b)
    except:
        return str(a) == str(b)

def is_within_percentage(a, b, percentage):
    """두 숫자가 지정된 백분율 이내인지 확인"""
    if pd.isna(a) or pd.isna(b) or a <= 0 or b <= 0:
        return False
    min_val = min(a, b)
    max_val = max(a, b)
    return ((max_val - min_val) / min_val) <= (percentage / 100)

def check_match_condition(mc_row, combined_row, step):
    """각 단계별 매칭 조건 확인"""
    if step == 1:  # 연면적 + 사용승인날짜 정확히 일치
        area_match = safe_equals(mc_row['연면적'], combined_row['연면적(㎡)'])
        mc_date = safe_date_convert(mc_row['사용승인연도'])  # 수정됨
        combined_date = safe_date_convert(combined_row['사용승인일'])
        date_match = mc_date == combined_date and mc_date is not None
        return area_match and date_match
    
    elif step == 2:  # 연면적 + 텍스트 토큰 일치
        area_match = safe_equals(mc_row['연면적'], combined_row['연면적(㎡)'])
        token_match = len(mc_row['mc_tokens'] & combined_row['combined_tokens']) > 0
        return area_match and token_match
    
    elif step == 3:  # 연면적 정확히 일치
        return safe_equals(mc_row['연면적'], combined_row['연면적(㎡)'])
    
    elif step == 4:  # 연면적 1% 이내 + 사용승인날짜 일치
        area_match = is_within_percentage(mc_row['연면적'], combined_row['연면적(㎡)'], 1)
        mc_date = safe_date_convert(mc_row['사용승인연도'])  # 수정됨
        combined_date = safe_date_convert(combined_row['사용승인일'])
        date_match = mc_date == combined_date and mc_date is not None
        return area_match and date_match
    
    elif step == 5:  # 연면적 1% 이내 + 텍스트 토큰 일치
        area_match = is_within_percentage(mc_row['연면적'], combined_row['연면적(㎡)'], 1)
        token_match = len(mc_row['mc_tokens'] & combined_row['combined_tokens']) > 0
        return area_match and token_match
    
    elif step == 6:  # 연면적 1% 이내 일치
        return is_within_percentage(mc_row['연면적'], combined_row['연면적(㎡)'], 1)
    
    elif step == 7:  # 텍스트 토큰 일치
        return len(mc_row['mc_tokens'] & combined_row['combined_tokens']) > 0
    
    return False

def match_universities():
    """대학교 매칭 수행"""
    print("=== 대학교 데이터 매칭 시작 ===")
    
    target_universities = ['서울대학교', '전북대학교', '부경대학교', '경북대학교', '부산대학교']
    
    # mc_df에 토큰 컬럼 추가
    mc_df['mc_tokens'] = mc_df['건축물명'].apply(tokenize_text)
    
    # combined_df에 토큰 컬럼 추가
    combined_df['combined_tokens'] = combined_df.apply(
        lambda row: tokenize_text(str(row['건물명']) + ' ' + str(row['동명칭'])), axis=1
    )
    
    matching_results = []
    total_matched = 0
    
    for university in target_universities:
        print(f"\n--- {university} 매칭 시작 ---")
        
        # 해당 대학교의 mc_df 데이터에서 MGM_BLD_PK가 비어있는 데이터만 필터링
        mc_univ_data = mc_df[mc_df['기관명'] == university].copy()
        empty_mgm_mask = mc_univ_data['MGM_BLD_PK'].isna() | (mc_univ_data['MGM_BLD_PK'] == '')
        mc_univ_data = mc_univ_data[empty_mgm_mask]
        
        print(f"{university} 매칭 대상: {len(mc_univ_data)}건")
        
        if len(mc_univ_data) == 0:
            print(f"{university}: 매칭할 데이터가 없습니다.")
            continue
        
        # 해당 대학교의 combined_df 데이터
        combined_univ_data = combined_df[combined_df['대학교명'] == university]
        print(f"{university} 후보군: {len(combined_univ_data)}건")
        
        if len(combined_univ_data) == 0:
            print(f"{university}: 후보군이 없습니다.")
            continue
        
        univ_matched = 0
        
        # 각 mc 데이터에 대해 매칭 시도
        for mc_idx, mc_row in mc_univ_data.iterrows():
            matched = False
            
            # 7단계 순차적 매칭
            for step in range(1, 8):
                candidates = []
                
                # 모든 후보군과 비교
                for combined_idx, combined_row in combined_univ_data.iterrows():
                    if check_match_condition(mc_row, combined_row, step):
                        candidates.append((combined_idx, combined_row))
                
                # 유일한 후보가 있으면 매칭
                if len(candidates) == 1:
                    combined_idx, combined_row = candidates[0]
                    
                    # 매칭 결과 저장
                    mc_df.at[mc_idx, 'new_MGM'] = combined_row['new_MGM']
                    mc_df.at[mc_idx, 'MGM_BLD_PK'] = combined_row['MGM_BLD_PK']
                    mc_df.at[mc_idx, 'matching_step'] = f"{step}차"
                    
                    # 매칭에 사용된 변수들 저장
                    if step in [1, 4]:  # 날짜 포함 단계
                        mc_df.at[mc_idx, 'matched_date'] = safe_date_convert(combined_row['사용승인일'])
                    if step in [2, 5, 7]:  # 토큰 포함 단계
                        matched_tokens = mc_row['mc_tokens'] & combined_row['combined_tokens']
                        mc_df.at[mc_idx, 'matched_tokens'] = ', '.join(matched_tokens)
                    
                    mc_df.at[mc_idx, 'matched_area'] = combined_row['연면적(㎡)']
                    
                    matching_results.append({
                        'university': university,
                        'mc_index': mc_idx,
                        'combined_index': combined_idx,
                        'step': step,
                        'mc_area': mc_row['연면적'],
                        'combined_area': combined_row['연면적(㎡)'],
                        'building_name': mc_row['건축물명']
                    })
                    
                    matched = True
                    univ_matched += 1
                    break
                
                elif len(candidates) > 1:
                    # 다중 후보가 있으면 다음 단계로
                    continue
            
            if not matched:
                print(f"매칭 실패: {mc_row['건축물명']} (연면적: {mc_row['연면적']})")
        
        print(f"{university} 매칭 완료: {univ_matched}건")
        total_matched += univ_matched
    
    print(f"\n=== 전체 매칭 완료 ===")
    print(f"총 매칭된 건수: {total_matched}건")
    
    # 단계별 매칭 결과 요약
    if 'matching_step' in mc_df.columns:
        step_summary = mc_df['matching_step'].value_counts().sort_index()
        print("\n=== 단계별 매칭 결과 ===")
        for step, count in step_summary.items():
            print(f"{step}: {count}건")
    
    return matching_results

# 매칭 실행
results = match_universities()

=== 대학교 데이터 매칭 시작 ===

--- 서울대학교 매칭 시작 ---
서울대학교 매칭 대상: 33건
서울대학교 후보군: 190건


C:\Users\HyeonPoong Lee\AppData\Local\Temp\ipykernel_18420\3001629456.py:139: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '102212795' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  mc_df.at[mc_idx, 'new_MGM'] = combined_row['new_MGM']


매칭 실패: 관악 12동[사범교육협력센터] (연면적: 5123.92)
매칭 실패: 122-3동(교수아파트3) (연면적: 5138.12)
매칭 실패: 062(중앙도서관) (연면적: 30505.46)
매칭 실패: 023(자연과학관5) (연면적: 4440.58)
매칭 실패: 024(자연과학관6) (연면적: 6962.64)
매칭 실패: 025(자연과학관7) (연면적: 6624.79)
매칭 실패: 027(자연과학관9) (연면적: 6359.34)
매칭 실패: 031(공학관2) (연면적: 6987.43)
매칭 실패: 133(자동화연구소) (연면적: 4898.45)
매칭 실패: 020(약학관3) (연면적: 6575.52)
매칭 실패: 003(인문관3) (연면적: 4523.63)
매칭 실패: 005(인문관4) (연면적: 5181.18)
매칭 실패: 20(치의학대학원 본관) (연면적: 12092.58)
서울대학교 매칭 완료: 20건

--- 전북대학교 매칭 시작 ---
전북대학교 매칭 대상: 47건
전북대학교 후보군: 185건
매칭 실패: 공과대학2호관 (연면적: 6421.6)
매칭 실패: 의과대학1호관 (연면적: 8303.0)
매칭 실패: 의과대학2호관 (연면적: 4591.26)
매칭 실패: 본부행정동(본관) (연면적: 3391.2)
매칭 실패: 사회대본관 (연면적: 5856.92)
매칭 실패: 농대2호관 (연면적: 5960.27)
매칭 실패: 농업과학기술연구소 (연면적: 4055.5)
전북대학교 매칭 완료: 40건

--- 부경대학교 매칭 시작 ---
부경대학교 매칭 대상: 6건
부경대학교 후보군: 58건
부경대학교 매칭 완료: 6건

--- 경북대학교 매칭 시작 ---
경북대학교 매칭 대상: 52건
경북대학교 후보군: 118건
매칭 실패: 9.기숙사A동 (연면적: 4455.73)
매칭 실패: 2.중앙도서관 (연면적: 27750.76)
경북대학교 매칭 완료: 50건

--- 부산대학교 매칭 시작 ---
부산대학교 매칭 대상: 40건
부산대학교 후보군: 110건
매칭 실패: 

In [21]:
# 각 대학교별 MGM_BLD_PK 비어있는 건수 상세 확인
target_universities = ['서울대학교', '전북대학교', '부경대학교', '경북대학교', '부산대학교']

print("=== 각 대학교별 MGM_BLD_PK 현황 ===")

for university in target_universities:
    # 해당 대학교 전체 데이터
    univ_total = mc_df[mc_df['기관명'] == university]
    total_count = len(univ_total)
    
    # MGM_BLD_PK가 비어있는 데이터
    empty_mgm = univ_total['MGM_BLD_PK'].isna() | (univ_total['MGM_BLD_PK'] == '')
    empty_count = empty_mgm.sum()
    
    # MGM_BLD_PK가 이미 채워져 있는 데이터  
    filled_count = total_count - empty_count
    
    # 이번에 매칭된 건수 (matching_step이 있는 것들)
    if 'matching_step' in mc_df.columns:
        newly_matched = len(univ_total[univ_total['matching_step'].notna()])
    else:
        newly_matched = 0
    
    # 여전히 비어있는 건수
    still_empty = empty_count - newly_matched
    
    print(f"\n{university}:")
    print(f"  - 전체 건수: {total_count}건")
    print(f"  - 기존에 채워진 건수: {filled_count}건")
    print(f"  - 매칭 전 비어있던 건수: {empty_count}건")
    print(f"  - 이번에 매칭된 건수: {newly_matched}건")
    print(f"  - 여전히 비어있는 건수: {still_empty}건")
    print(f"  - 매칭률: {newly_matched/empty_count*100:.1f}%" if empty_count > 0 else "  - 매칭률: 0%")

# 전체 요약
total_all = len(mc_df[mc_df['기관명'].isin(target_universities)])
total_empty = len(mc_df[(mc_df['기관명'].isin(target_universities)) & 
                       (mc_df['MGM_BLD_PK'].isna() | (mc_df['MGM_BLD_PK'] == ''))])
total_matched = len(mc_df[(mc_df['기관명'].isin(target_universities)) & 
                         (mc_df['matching_step'].notna())]) if 'matching_step' in mc_df.columns else 0

print(f"\n=== 전체 요약 ===")
print(f"5개 대학교 전체 건수: {total_all}건")
print(f"매칭 전 총 빈 건수: {total_empty}건")
print(f"이번에 매칭된 건수: {total_matched}건")
print(f"여전히 빈 건수: {total_empty - total_matched}건")
print(f"전체 매칭률: {total_matched/total_empty*100:.1f}%" if total_empty > 0 else "전체 매칭률: 0%")

=== 각 대학교별 MGM_BLD_PK 현황 ===

서울대학교:
  - 전체 건수: 124건
  - 기존에 채워진 건수: 111건
  - 매칭 전 비어있던 건수: 13건
  - 이번에 매칭된 건수: 20건
  - 여전히 비어있는 건수: -7건
  - 매칭률: 153.8%

전북대학교:
  - 전체 건수: 66건
  - 기존에 채워진 건수: 59건
  - 매칭 전 비어있던 건수: 7건
  - 이번에 매칭된 건수: 40건
  - 여전히 비어있는 건수: -33건
  - 매칭률: 571.4%

부경대학교:
  - 전체 건수: 35건
  - 기존에 채워진 건수: 35건
  - 매칭 전 비어있던 건수: 0건
  - 이번에 매칭된 건수: 6건
  - 여전히 비어있는 건수: -6건
  - 매칭률: 0%

경북대학교:
  - 전체 건수: 73건
  - 기존에 채워진 건수: 71건
  - 매칭 전 비어있던 건수: 2건
  - 이번에 매칭된 건수: 50건
  - 여전히 비어있는 건수: -48건
  - 매칭률: 2500.0%

부산대학교:
  - 전체 건수: 56건
  - 기존에 채워진 건수: 54건
  - 매칭 전 비어있던 건수: 2건
  - 이번에 매칭된 건수: 38건
  - 여전히 비어있는 건수: -36건
  - 매칭률: 1900.0%

=== 전체 요약 ===
5개 대학교 전체 건수: 354건
매칭 전 총 빈 건수: 24건
이번에 매칭된 건수: 154건
여전히 빈 건수: -130건
전체 매칭률: 641.7%


In [24]:
# 엑셀 파일로 저장
mc_df.to_excel('대학교_매칭_결과.xlsx', index=False)
print("엑셀 파일이 저장되었습니다: 대학교_매칭_결과.xlsx")

엑셀 파일이 저장되었습니다: 대학교_매칭_결과.xlsx
